<a href="https://colab.research.google.com/github/matthieudeReynal/PI2A5-CryptoCurrenciesTrading/blob/master/Copie_de_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import de librairies

In [1]:
import requests # requetes web GET 
import pandas as pd
import pathlib # gestion des chemin d'accès fichier
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from collections import Counter

import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import HuberRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit

from sklearn.utils import all_estimators
from sklearn.linear_model import ElasticNet
from sklearn.externals import joblib
from flask import Flask, jsonify, request


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


#Importation des données

In [2]:

import matplotlib.pyplot as plt
import numpy as np
from dataToDictionnary import *
import pandas as pd
pd.options.mode.chained_assignment = None

pair = 'SNX/USDT'
exchange = ['binance']
duration = ['3m']
df = oneAssetToDFalex(pair, duration, exchange)
df['date'] = pd.to_datetime(df['date'],unit='ms'à)
print(df.head())
#df = df.iloc[80800:81000]
#df = df.reset_index(drop=True)
print(df)
# In[2]:

#the calculation of sar
#as rules are very complicated
#plz check the links above to understand more about it


SyntaxError: ignored

# Preparation des données

In [ ]:
#df.sort_values(by=['col1'])
df=shuffle(df)
dfTrain = df[:60000]
dfTest = df[60000:]
#dfTrain.sort_index()
#dfTest.sort_index()

X_Train=dfTrain.loc[:,df.columns != 'result']
Y_Train=dfTrain.loc[:,'result]
X_Test=dfTest.loc[:,df.columns != 'result']
Y_Test=dfTest.loc[:,'result']


# modèles de machine learning
Pour chacun des modèles nous faisons un gridsearch dont le but de tester le modele avec plusieurs parametres. 

Ensuite nous affichons et classons les parametres pour ne selectionner que les meilleurs.

Nous reproduisons ensuite le meilleur modele afin de l'enregistrer. 
Nous affichons à ce moment le taux de réussite du modele.

Le meilleur modele est enfin sélectionné pour notre API. Dans notre cas il s'agit du random Forest.

La regression linéaire est le seul modele qui n'a pas de grids search, il n'y  pas de paramettre important à tester

In [ ]:
 def grid_search(X, Y, model, param_grid={}):
    rk = {'random_state':42}
    try:
        model(**rk)
    except:
        rk = {}
        
    grid = GridSearchCV(
        model(**rk),
        param_grid=param_grid,
        cv=ShuffleSplit(n_splits=4, random_state=20),
        verbose=0,
        n_jobs=-1
    )

    grid.fit(X, Y)
    return grid

## Random Forest Regressor
Le random Forest ajuste un certain nombre de classificateurs d'arbres de décision sur divers sous-échantillons de l'ensemble de données et utilise la moyenne pour améliorer la précision de la prédiction.

Ces calcul peuvent etre très très long, je les ai fait,cela a pris environ 1h30. Voila la grille testé complete. Seul certain parametre incluant les plus performant ont été gardé.

max_depth': [None, 10,20, 50, 100]
n_estimators': [10,50, 100, 200]
criterion': ['mse','mae']
'max_features' : ['auto','sqrt', 'log2']

In [ ]:
modelRandomForest = RandomForestRegressor

param_grid = {
    'max_depth': [None, 10,20, 50, 100],
    'criterion': ['mse'],
    'n_estimators': [10,50,100],
    'max_features' : ['auto'] 
   
}
useful_columns = ['param_' + k for k in param_grid.keys()] + ['mean_test_score', 'std_test_score', 'rank_test_score']
grid = grid_search(X_Train, Y_Train, modelRandomForest, param_grid)
pd.DataFrame(grid.cv_results_)[useful_columns].sort_values('rank_test_score')

In [ ]:
modelRandomForest = RandomForestRegressor(max_depth=None ,criterion='mse',n_estimators=100,max_features='auto').fit(X_Train,Y_Train)
Y_Prediction=modelRandomForest.predict(X_Test)

Y_PredictionBinarie=[]
for i in range(len(Y_Prediction)):
    if Y_Prediction[i]>0.5:
        Y_PredictionBinarie.append(1)
    else:
       Y_PredictionBinarie.append(0)

counter=0
iterator=0
for i, value in Y_Test.iteritems():

    if(int(Y_PredictionBinarie[iterator])==int(value)):
       counter=counter+1
    iterator=iterator+1

print("Taux de bons resultats: ",counter/8,'%')

In [ ]:
plt.scatter(X_Test.index, Y_Test,color='black', s=1)
plt.scatter(X_Test.index, Y_Prediction, color='blue', s=1)

plt.xticks(())
plt.yticks(())

plt.show()